In [ ]:
%pip install transformers sklearn datasets ipywidgets

In [ ]:
'''
import tensorflow as tf
from tensorflow import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(2)
'''

from tqdm import tqdm
import numpy as np

# Challenge 3: Sentiment analysis
***
We are interested in predicting the sentiment of written text.
* For the challenge, we adopt the IMDb dataset of movie reviews:
    * "[...] *this film is very lovable in a way many comedies are not* '[...]"

The task is simple:
* Predict whether a review has a positive or a negative sentiment
    * Input: Paragraphs of text (string) and binary label (1: positive, 0: negative)
    * Metric: Accuracy
* Examples to get you started:
    * Finetune transformer models, e.g. BERT
    * Word2Vec + Deep Neural Network of your choice

# Hints
***
When training transformers:
* The models are *huge*, so training will run very slowly
    * Running several batches of the full training data will be a costly operation
    * Google Colab needs you to do a captcha every 2h...


# Loading the data
***
Using Huggingface `datasets` library:

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("imdb")
print(raw_datasets["train"]["text"][2])

# Dataset preprocessing
***
The function below translates the sentences to token IDs and splits into train and validation:

In [ ]:
def tokenize_and_split(datasets, tokenizer):

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = datasets.map(tokenize_function, batched=True)
    train_dataset = tokenized_datasets["train"]
    eval_dataset = tokenized_datasets["test"]
    return train_dataset, eval_dataset

def as_tf_dataset(dataset, tokenizer, batch_size=8):
    tf_data = dataset.remove_columns(["text"]).with_format("tensorflow")
    train_features = {x: tf_data[x].to_tensor() for x in tokenizer.model_input_names}
    tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_data["label"]))
    tf_dataset = tf_dataset.shuffle(len(tf_dataset)).batch(batch_size)
    return tf_dataset

# Finetuning transformers
***
Huggingface's `transformers` library provides excellent functionality and many pretrained models
* Let's load a smaller variant of BERT, called `DistilBert` and its tokenizer

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Run tokenization
***
Apply tokenizer and sample a small subset for showcasing the workflow

In [ ]:
train_data, test_data = tokenize_and_split(raw_datasets, tokenizer)
small_train_dataset = train_data.shuffle(seed=42).select(range(1000))
small_eval_dataset = test_data.shuffle(seed=42).select(range(1000))

# Variant 1: Train using `keras`
***
We need to convert to a dataset format that `keras` understands:

In [ ]:
tf_train_small = as_tf_dataset(small_train_dataset, tokenizer, batch_size=8)
tf_eval_small = as_tf_dataset(small_eval_dataset, tokenizer, batch_size=8)
print(tf_train_small)

# Loading the model
***

In [ ]:
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.summary(50)

# Time to train (`keras`)
***

In [ ]:
from tensorflow import keras

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-5),
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_small, validation_data=tf_eval_small, epochs=3, verbose=1)

# Evaluation code for `keras`
***
We run the final evaluation on the full test dataset:

In [ ]:
tf_eval_full = as_tf_dataset(test_data, tokenizer, batch_size=16)

In [ ]:
loss, acc = model.evaluate(tf_eval_full)
print(f"Reached {acc:.3f} accuracy and a loss of {loss:.4f}")

# Variant 2: Train using PyTorch
***
The `transformers` framework comes with its own `Trainer` class which we can use

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Specifying training arguments
***
The usual hyperparameters can be set using `TrainingArguments`

In [ ]:
training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    evaluation_strategy="epoch",     # when to run evaluation
)

# Adding accuracy metric
***
Evaluation workflow is a little different to what we are used from `keras`:

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Time to train (PyTorch)
***

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_dataset,   # training dataset
    eval_dataset=small_eval_dataset,     # evaluation dataset
    compute_metrics=compute_metrics,     # code to run accuracy metric
)
trainer.train()

# Evaluation code for PyTorch
***
We simply define a new `Trainer` that runs on the complete `eval_data`

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,
)
results = trainer.evaluate()
loss, acc = results["eval_loss"], results["eval_accuracy"]
print(f"Reached {acc:.3f} accuracy and a loss of {loss:.4f}")

# Applying Word2Vec
***
The previous state-of-the-art NLP models made heavy use of Word2Vec embeddings:
1. Learn a word embedding on a large amount of text (unsupervised)
    * It's also possible to train on task-specific text only
2. Translate all words in an input sequence to their vectors
3. Apply sequences of word vectors to a network model of your choice
4. ???
5. Profit.

In [ ]:
%pip install gensim

# Getting pre-trained vectors
***
`gensim` is a popular framework for training Word2Vec models. It also has a model zoo:

In [ ]:
import gensim
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))
# random pick:
w2v = gensim.downloader.load('word2vec-google-news-300')

Here is the famous `king - man + woman = queen` example:

In [ ]:
w2v.most_similar(positive=["woman", "king"], negative=["man"])

# Data preprocessing
***
For starters, we do the simplest possible tokenization: Splitting by `" "`

In [ ]:
def tokenize_w2v(w2v, dataset, sentence_function=None):
    '''
        sentence_function will be applied to each list of translated vectors.
        This allows to save a lot of RAM when wishing to aggregate the paragraphs.
    '''
    x = []
    y = np.array(dataset["label"], "int32")
    for text in tqdm(dataset["text"]):
        paragraph = [w2v[token] for token in text.split(" ") if token in w2v]
        if sentence_function is None:
            paragraph = np.array(paragraph, "float32")
        else: 
            paragraph = sentence_function(paragraph)
        x.append(paragraph)
    return x, y

# Getting rid of variable length data
***
For a first simple prototype, we simply sum up all word vectors of a review
* Per review, we will get a single vector
    * But is that a good approach?

In [ ]:
sum_of_words = lambda vectors: np.sum(vectors, axis=0)
x_train, y_train = tokenize_w2v(w2v, raw_datasets["train"], sum_of_words)
x_train = np.array(x_train)
x_test, y_test = tokenize_w2v(w2v, raw_datasets["test"],sum_of_words)
x_test = np.array(x_test)

In [ ]:
# shuffle training samples
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

# Training with summed word vectors
***


In [ ]:
from tensorflow.keras.layers import (
    Input, Dense, Dropout,
)
from tensorflow.keras.models import Model

input_layer = Input(w2v.vector_size)
l = input_layer
l = Dense(128, "relu")(l)
l = Dropout(0.4)(l)
l = Dense(1, "sigmoid")(l)
model = Model(input_layer, l)
model.summary(50)

In [ ]:
opt = keras.optimizers.Adam(lr=0.01)
model.compile(opt, "binary_crossentropy", ["accuracy"])
model.fit(x_train, y_train, batch_size=128, epochs=1000,
          verbose=2, validation_split=0.2)

# Evaluation code for W2V based models
***

In [ ]:
loss, acc = model.evaluate(x_test, y_test, batch_size=128)
print(f"Reached {acc:.3f} accuracy and a loss of {loss:.4f}")